In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np

Mounted at /content/drive


In [ ]:
# Step 1: Load the trained model
model_path = '/content/drive/MyDrive/30000/best_30000_17db_model.h5'
model = load_model(model_path)

# Step 2: Load training and validation data
X_train = np.load('/content/drive/MyDrive/30000/merged_signals_30000.npy')
y_train = np.load('/content/drive/MyDrive/30000/merged_labels_30000.npy')

# Split data into training and validation sets
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

print(f"Training Data Shape: {X_train.shape}")
print(f"Validation Data Shape: {X_val.shape}")

Training Data Shape: (48000, 36, 1000, 2)
Validation Data Shape: (12000, 36, 1000, 2)


In [ ]:
# Step 3: Convert the model to TensorFlow Lite format with BF16 quantization
def convert_to_tflite_bf16(model, X_train):
    # Define the representative dataset generator
    def representative_dataset_generator():
        for i in range(len(X_train)):
            yield [np.expand_dims(X_train[i], axis=0)]

    # Convert the model to TensorFlow Lite format with BF16 quantization
    converter = tf.lite.TFLiteConverter.from_keras_model(model)
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.representative_dataset = representative_dataset_generator
    converter.target_spec.supported_types = [tf.float16]

    tflite_model = converter.convert()
    return tflite_model

print("Converting the model to TensorFlow Lite format with BF16 quantization...")
bf16_tflite_model = convert_to_tflite_bf16(model, X_train)

Converting the model to TensorFlow Lite format with BF16 quantization...


In [ ]:
# Step 4: Save the converted TFLite model
tflite_model_path = '/content/drive/MyDrive/30000/best_30000_17db_model_bf16.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(bf16_tflite_model)

print(f"BF16 quantized model saved to {tflite_model_path}")

BF16 quantized model saved to /content/drive/MyDrive/30000/best_30000_17db_model_bf16.tflite


In [ ]:
# Step 5: Evaluate the BF16 quantized TFLite model
def evaluate_tflite_model(tflite_model_path, X_val, y_val):
    # Load the TFLite model and allocate tensors
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    correct_predictions = 0

    # Perform inference on the validation set
    for i in range(len(X_val)):
        input_data = np.expand_dims(X_val[i], axis=0).astype(input_details[0]['dtype'])
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        output_data = interpreter.get_tensor(output_details[0]['index'])

        # Apply threshold to classify
        predicted_label = (output_data >= 0.5).astype(int)
        true_label = y_val[i]

        if len(true_label.shape) > 1:
            true_label = np.argmax(true_label)

        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(X_val)
    return accuracy

print("Evaluating the BF16 quantized TFLite model...")
bf16_accuracy = evaluate_tflite_model(tflite_model_path, X_val, y_val)
print(f"Accuracy of the BF16 quantized model: {bf16_accuracy:.4f}")

Evaluating the BF16 quantized TFLite model...
Accuracy of the BF16 quantized model: 0.7522
